In [2]:
!git clone https://github.com/mnielsen/neural-networks-and-deep-learning.git

Cloning into 'neural-networks-and-deep-learning'...
fatal: unable to access 'https://github.com/mnielsen/neural-networks-and-deep-learning.git/': Could not resolve host: github.com


In [3]:
import numpy as np

In [11]:
class Network:
    """ This class Represents the whole Neural Netwrk"""
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.rand(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))

In [5]:
n = Network([2,3,1])

In [10]:
n.weights

[array([[0.27395704, 0.74468463],
        [0.54822895, 0.7652089 ],
        [0.75051708, 0.35320003]]),
 array([[0.45058184, 0.90089806, 0.54615301]])]